In [9]:
from decoder.geoutils import *
from decoder import Decoder
from rich import print
import numpy as np

In [10]:
decoder = Decoder()
data = decoder.load(
    "Test_Data/datos_asterix_radar.ast",
    parallel=False,
    max_messages=10,
)
print(data[1])

Loaded 30060083 bytes from Test_Data/datos_asterix_radar.ast

Decoding: 100%|██████████| 11/11 [00:00<00:00, 2084.93Msg/s]

{
    'Category': 48,
    'SAC': 20,
    'SIC': 129,
    'Time': '4.0:0.0:0.828125',
    'Target_Type': 'Mode S Roll-Call + PSR',
    'Simulated': False,
    'RDP': False,
    'SPI': False,
    'RAB': False,
    'Range NM': 1.69921875,
    'Range m': 3146.953125,
    'Theta': 215.6231689453125,
    'Mode3A': 1000,
    'FL': 2.25,
    'Height (ft)': 225.0,
    'Height (m)': 68.58,
    'SSR plot runlength': 1.3623046875,
    'Number of received replies SSR': 5,
    'Amplitude of (M)SSR reply': 227,
    'Primary Plot Runlength (deg)': 0.7470703125,
    'Amplitude of Primary Plot (dBm)': 47,
    'Range (PSR-SSR)': 0.109375,
    'Azimuth (PSR-SSR)': 5.537109375,
    'TA': '34310D',
    'TI': 'BCS3LR',
    'Repetition': 2,
    '4,0': {
        'Status MCP/FCU': True,
        'MCP/FCU Selected Altitude': 200.3125,
        'Status FMS': True,
        'FMS Selected Altitude': 204.125,
        'Status Barometric Reference': True,
        'Barometric Pressure Setting': 902.4,
        'Status MCP/FCU Mode': False,
        'VNAV Mode': False,
        'ALT Hold Mode': True,
        'Approach Mode': False,
        'Status Target Source': False,
        'Target Alt Source': 'Unknown'
    },
    '6,0': {
        'Status Magnetic Heading': False,
        'Magnetic Heading': -19.6875,
        'Status IAS': True,
        'IAS': 134.0,
        'Status Mach': False,
        'Mach': 3.064,
        'Status Barometric Altitude Rate': False,
        'Barometric Altitude Rate': 16288.0,
        'Status Inertial Vertical Velocity': True,
        'Inertial Vertical Velocity': 11264.0
    },
    'Groundspeed (kt)': 126.5,
    'Heading (deg)': 18.4295654296875,
    'ConfVTent': False,
    'Type of Sensor': 'Combined Track',
    'DOU': False,
    'Manoeuver detection Horizontal': False,
    'Climbing/Descending': 'Unknown',
    'Communications Capability': 'Comm A and B',
    'Flight Status': 'No alert, no SPI, airborne',
    'SI/II': 'SI',
    'Mode S Specific Service Capability': True,
    'Altitude Reporting Capability': True,
    'Aircraft Identification Capability': True,
    'ACAS Status': 'Operational',
    'Hybrid Surveillance': False,
    'TA/RA': 'TA',
    'Applicable MOPS Doc': 'RTCA DO-185A'
}

In [18]:
rho = data[1]["Range m"]
theta = data[1]["Theta"] * np.pi / 180
H = data[1]["Height (m)"]

radar_lat = (41 + 18 / 60.0 + 2 / 3600.0)*np.pi/180
radar_lon = (2 + 6 / 60.0 + 7 / 3600.0)*np.pi/180
radar_alt = 27
coords_radar = CoordinatesWGS84(radar_lat, radar_lon, radar_alt)

Rt=6378e3

In [19]:
elevation = np.asin((H - radar_alt) / rho)
coords_polar = CoordinatesPolar(rho, theta, elevation)

In [20]:
coords_cartesian = GeoUtils.change_radar_spherical_2_radar_cartesian(coords_polar)

In [21]:
coords_geocentric = GeoUtils().change_radar_cartesian_2_geocentric(
    coords_radar, coords_cartesian
)

In [22]:
coords_geodesic = GeoUtils().change_geocentric_2_geodesic(coords_geocentric)
print(str(coords_geodesic))

41:16:40.0837N 002:04:48.2477E 96.5015m
lat:41.277801024 lon:2.080068817

In [24]:
b=Rt+H
c=Rt+radar_alt
alpha=np.arccos((b**2+c**2-rho**2)/(2*b*c))
print("alpha (deg): ", alpha*180/np.pi)
coords_radar = CoordinatesWGS84(radar_lat, radar_lon, radar_alt)
print("Radar coords (lat, lon, alt):",radar_lat, radar_lon, radar_alt)
print(str(coords_radar))
coords_p=CoordinatesWGS84(radar_lat+alpha*np.sin(theta),radar_lon+alpha*np.cos(theta),H)
print(str(coords_p))
print("New point coords (lat, lon, alt):",radar_lat+alpha*180*np.sin(theta)/np.pi,radar_lon+alpha*180*np.cos(theta)/np.pi,H)

alpha (deg):  0.028267488844797697

Radar coords (lat, lon, alt): 0.7208306773472802 0.03668585124955859 27

41:18:2.0000N 002:06:7.0000E 27.0000m
lat:41.300555556 lon:2.101944444

41:17:2.7280N 002:04:44.2804E 68.5800m
lat:41.284091108 lon:2.078966784

New point coords (lat, lon, alt): 0.7043662298588637 0.013708190478889694 68.58